In [3]:
import sys
import os
sys.path.append(os.path.abspath('../input/sentence-transformers'))

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
import pandas as pd
df = pd.read_csv('../input/diffusiondb-filtered-0-40000/diffusiondb.csv')

In [9]:
df.iloc[0]['filepath']

'/kaggle/input/diffusiondb-filtered-0-40000/0-40000/2217ccbd-a1c6-47ac-9a2d-79649727c834.png'

In [1]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")

/home/jaehoon_scatterlab_co_kr/projects/image-to-prompts/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model.config

GPT2Config {
  "_name_or_path": "Gustavosta/MagicPrompt-Stable-Diffusion",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.27.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [12]:
from transformers import GPT2Config, GPT2Tokenizer
model_config = GPT2Config("Gustavosta/MagicPrompt-Stable-Diffusion")
tokenizer = GPT2Tokenizer.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")

In [13]:
tokenizer("My name is Julien and I like to")

{'input_ids': [3666, 1438, 318, 5979, 2013, 290, 314, 588, 284], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer.eos_token_id

50256

In [55]:
model = GPT2LMHeadModel.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion", pad_token_id=tokenizer.eos_token_id)

In [85]:
import torch
#t = tokenzier("My name is Julien and I like to")
inputs = tokenizer("ramen carved out of fractal rose ebony", return_tensors="pt")
generation_output = model.generate(**inputs, do_sample=True, top_p=0.5, max_length=77, no_repeat_ngram_size=5)
#generation_output = model.generate(**inputs, num_beams=5, max_length=50, early_stopping=True)

In [86]:
out = tokenizer.decode(generation_output.tolist()[0], skip_special_tokens=True)
print(out)

ramen carved out of fractal rose ebony, carved out of sapphire gems, inlaid with ivory and gold accents, ivory rococo, wings lace wear, sculpted by spider zero, zaha hadid, trending on artstation, beautifully lit, hyper detailed, insane details, intricate, elite, ornate, elegant, luxury + + upward perspective +


In [80]:
embedding = model.get_input_embeddings()
input_embeddings = embedding(inputs['input_ids'])

In [82]:
input_embeddings.shape

torch.Size([1, 10, 768])

In [26]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Config

class CustomGPT2LMHeadModel(GPT2LMHeadModel):
    def __init__(self, config):
        super(CustomGPT2LMHeadModel, self).__init__(config)
        self.image_projector = nn.Linear(config.image_embedding_size, config.n_embd)

    def forward(
        self,
        input_ids=None,
        image_embeddings=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        # Project image embeddings to token embeddings
        if image_embeddings is not None:
            image_token_embeddings = self.image_projector(image_embeddings)
            inputs_embeds = image_token_embeddings if inputs_embeds is None else torch.cat([inputs_embeds, image_token_embeddings], dim=1)

        return super().forward(
            input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            encoder_hidden_states=encoder_hidden_states,
            encoder_attention_mask=encoder_attention_mask,
            labels=labels,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
    
    def generate(
        self,
        input_ids=None,
        image_embeddings=None,
        max_length=None,
        **generate_kwargs
    ):
        # Project image embeddings to token embeddings
        if image_embeddings is not None:
            image_token_embeddings = self.image_projector(image_embeddings)
            inputs_embeds = image_token_embeddings if input_ids is None else torch.cat([self.wte(input_ids), image_token_embeddings], dim=1)
            input_ids = None

        return super().generate(
            input_ids=input_ids,
            max_length=max_length,
            **generate_kwargs
        )


In [27]:
config = GPT2Config.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")
config.image_embedding_size = 1024  # Set this to the size of your image embeddings


In [28]:
model = CustomGPT2LMHeadModel.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion", config=config)

Some weights of CustomGPT2LMHeadModel were not initialized from the model checkpoint at Gustavosta/MagicPrompt-Stable-Diffusion and are newly initialized: ['image_projector.weight', 'image_projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.config

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "image_embedding_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.27.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [2]:
from transformers import CLIPVisionConfig, CLIPVisionModel
config = CLIPVisionConfig.from_pretrained("openai/clip-vit-large-patch14")
vision_model = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14", config=config)

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.2.mlp.fc2.bias', 'text_model.encoder.layers.2.mlp.fc1.bias', 'text_model.encoder.layers.9.mlp.fc1.bias', 'text_model.encoder.layers.7.layer_norm2.weight', 'text_model.encoder.layers.6.self_attn.out_proj.weight', 'text_model.encoder.layers.2.self_attn.v_proj.bias', 'text_model.encoder.layers.8.layer_norm1.bias', 'text_model.encoder.layers.6.self_attn.k_proj.weight', 'text_model.encoder.layers.5.self_attn.v_proj.bias', 'text_model.encoder.layers.11.self_attn.k_proj.weight', 'text_model.encoder.layers.6.self_attn.q_proj.weight', 'text_model.encoder.layers.7.layer_norm1.bias', 'text_model.encoder.layers.8.self_attn.out_proj.weight', 'text_model.encoder.layers.11.mlp.fc2.bias', 'text_model.encoder.layers.4.layer_norm2.weight', 'text_model.encoder.layers.4.self_attn.v_proj.bias', 'text_model.encoder.layers.7.mlp.fc2.weight', 'text_model.encoder.la

In [3]:
config

CLIPVisionConfig {
  "_name_or_path": "openai/clip-vit-large-patch14",
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 1024,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 14,
  "projection_dim": 768,
  "transformers_version": "4.27.1"
}

In [7]:
from transformers import CLIPProcessor
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

In [8]:
import numpy as np
image = np.random.rand(3,224,224)
input_ = processor.image_processor(images=image)

In [9]:
import torch
out = vision_model(torch.tensor(input_['pixel_values']))

/tmp/ipykernel_31149/2158643839.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  out = vision_model(torch.tensor(input_['pixel_values']))


In [10]:
out.pooler_output

tensor([[-0.4104,  0.7516,  0.9528,  ..., -1.1397, -1.4692,  0.7188]],
       grad_fn=<NativeLayerNormBackward0>)

In [29]:
o = model.generate(image_embeddings=out.pooler_output, max_length=77)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [31]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")
tokenizer.decode(o.tolist()[0], skip_special_tokens=True)

'a beautiful painting of a cyberpunk skull by sergey kolesov and pascal blanche and rhads. in style of colorful comic noir illustration, symmetry, sci fi, hyper detailed. octane render. trending on artstation\n'

In [16]:
model.transformer

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (1): GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwis

In [1]:
import os,sys
sys.path.append(os.path.abspath("../"))
import json

from torch.utils.data import DataLoader
from models.dataset import ImageTextDataset, get_transformation_for_train
from transformers import GPT2Tokenizer

lm_name = 'gpt2-medium'
tokenizer = GPT2Tokenizer.from_pretrained(lm_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens(["<IMG>"])

train_data_path = "../datasets/train/metadata-v2.jsonl"
with open(train_data_path) as f:
    train_data = [json.loads(d) for d in f.readlines()]
image_size = (224,224)
max_seq_length=48
n_image_token=4
train_transform = get_transformation_for_train(image_size)
train_dataset = ImageTextDataset(
    train_data,
    tokenizer,
    max_seq_length=max_seq_length,
    n_image_token=n_image_token,
    transform=train_transform,
    image_size=image_size,
    root_path="/home/jaehoon_scatterlab_co_kr/projects/image-to-prompts/datasets/train",
)
batch_size=4
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
)

/home/jaehoon_scatterlab_co_kr/projects/image-to-prompts/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jaehoon_scatterlab_co_kr/projects/image-to-prompts/venv/lib/python3.8/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [9]:
item = train_dataset[1]

In [10]:
tokenizer.decode(item['input_ids'].tolist())

'<IMG> <IMG> <IMG> <IMG> amsterdam canals, sharp focus, key anime visuals, studio ghibli, trending on artstation, very detailed<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>'

In [7]:
item['input_ids'].tolist()

[50257,
 50257,
 50257,
 50257,
 41082,
 496,
 319,
 262,
 1735,
 286,
 257,
 19516,
 11,
 7786,
 2962,
 11,
 1994,
 11984,
 27329,
 11,
 8034,
 308,
 3145,
 4528,
 11,
 29056,
 319,
 1242,
 17529,
 11,
 6496,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256,
 50256]